In [1]:
# Install required libraries
import sys
!{sys.executable} -m pip install requests urllib3 pandas --quiet


In [2]:
import requests
import json
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import socket
from ipaddress import IPv4Network
import getpass
import pandas as pd

# Disable SSL warnings (since we're exploring local network)
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Pretty print helper
def pretty_print(data, max_depth=3, indent=0):
    """Pretty print nested dictionaries"""
    if isinstance(data, dict):
        for key, value in list(data.items())[:20]:  # Limit to first 20 items
            print('  ' * indent + f"{key}: ", end='')
            if isinstance(value, (dict, list)) and indent < max_depth:
                print()
                pretty_print(value, max_depth, indent + 1)
            else:
                print(str(value)[:100])  # Truncate long values
        if len(data) > 20:
            print(f'  ... ({len(data) - 20} more keys)')
    elif isinstance(data, list):
        for i, item in enumerate(data[:10]):  # Limit to first 10 items
            print('  ' * indent + f"[{i}]: ", end='')
            if isinstance(item, (dict, list)) and indent < max_depth:
                print()
                pretty_print(item, max_depth, indent + 1)
            else:
                print(str(item)[:100])
        if len(data) > 10:
            print(f'  ... ({len(data) - 10} more items)')


## Step 1: Discover the device IP

Let's try to figure out what `.1` means on your network - common possibilities:
- 192.168.1.1
- 10.0.0.1
- 172.16.0.1
- 192.168.0.1


In [3]:
# Get local network interface to determine likely network
import subprocess

def get_local_ip():
    """Get local IP address to infer network"""
    try:
        # Try to connect to a known address to determine local network
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception as e:
        print(f"Error getting local IP: {e}")
        return None

local_ip = get_local_ip()
print(f"Your local IP: {local_ip}")

# Common .1 addresses to try
possible_ips = []
if local_ip:
    parts = local_ip.split('.')
    if len(parts) == 4:
        # Common patterns
        possible_ips = [
            f"{parts[0]}.{parts[1]}.{parts[2]}.1",  # Same subnet .1
            "192.168.1.1",
            "10.0.0.1",
            "192.168.0.1",
            "172.16.0.1"
        ]
else:
    possible_ips = ["192.168.1.1", "10.0.0.1", "192.168.0.1", "172.16.0.1"]

# Remove duplicates while preserving order
possible_ips = list(dict.fromkeys(possible_ips))
print(f"\nWill try these IPs: {possible_ips}")


Your local IP: 192.168.1.101

Will try these IPs: ['192.168.1.1', '10.0.0.1', '192.168.0.1', '172.16.0.1']


In [4]:
# Test connectivity to possible IPs
def test_connectivity(ip, ports=[443, 80, 8443]):
    """Test if device is reachable on common ports"""
    results = {}
    for port in ports:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex((ip, port))
            sock.close()
            if result == 0:
                results[port] = True
            else:
                results[port] = False
        except Exception as e:
            results[port] = False
    return results

print("Testing connectivity...")
for ip in possible_ips:
    print(f"\n{ip}:")
    results = test_connectivity(ip)
    for port, reachable in results.items():
        status = "✓ OPEN" if reachable else "✗ CLOSED"
        print(f"  Port {port}: {status}")


Testing connectivity...

192.168.1.1:
  Port 443: ✓ OPEN
  Port 80: ✓ OPEN
  Port 8443: ✓ OPEN

10.0.0.1:
  Port 443: ✗ CLOSED
  Port 80: ✗ CLOSED
  Port 8443: ✗ CLOSED

192.168.0.1:
  Port 443: ✗ CLOSED
  Port 80: ✗ CLOSED
  Port 8443: ✗ CLOSED

172.16.0.1:
  Port 443: ✗ CLOSED
  Port 80: ✗ CLOSED
  Port 8443: ✗ CLOSED


## Step 2: Try to discover API endpoints (no auth)

Let's try hitting common UniFi endpoints without authentication first to see what responds.


In [5]:
# Common UniFi API endpoints to try
common_endpoints = [
    "/api/self",
    "/api/stat/system",
    "/api/system",
    "/api/info",
    "/api/auth/login",
    "/api/login",
    "/status",
    "/",
    "/api/s/default/self",
    "/api/s/default/stat/system",
    "/api/s/default/stat/device",
    "/api/s/default/stat/sta",
    "/proxy/network/api/s/default/stat/system",
    "/proxy/network/api/login",
]

def try_endpoint(ip, port, endpoint):
    """Try to hit an endpoint and see what responds"""
    protocols = ['https', 'http'] if port == 443 else ['http']
    if port == 8443:
        protocols = ['https']
    
    for protocol in protocols:
        url = f"{protocol}://{ip}:{port}{endpoint}"
        try:
            response = requests.get(url, verify=False, timeout=3, allow_redirects=False)
            return {
                'url': url,
                'status': response.status_code,
                'headers': dict(response.headers),
                'content_preview': response.text[:500],
                'content_type': response.headers.get('Content-Type', 'unknown')
            }
        except requests.exceptions.RequestException as e:
            continue
    return None

# Try endpoints on discovered IPs
discovered_apis = {}
for ip in possible_ips:
    ports_to_try = []
    conn_test = test_connectivity(ip)
    for port, reachable in conn_test.items():
        if reachable:
            ports_to_try.append(port)
    
    if ports_to_try:
        print(f"\n\n=== Trying {ip} ===")
        for port in ports_to_try:
            print(f"\nPort {port}:")
            for endpoint in common_endpoints[:5]:  # Try first 5 endpoints
                result = try_endpoint(ip, port, endpoint)
                if result and result['status'] < 500:
                    print(f"  {endpoint}: Status {result['status']}")
                    if result['status'] == 200:
                        discovered_apis[f"{ip}:{port}{endpoint}"] = result
                        print(f"    Content-Type: {result['content_type']}")
                        print(f"    Preview: {result['content_preview'][:100]}...")




=== Trying 192.168.1.1 ===

Port 443:
  /api/self: Status 401
  /api/stat/system: Status 401
  /api/system: Status 200
    Content-Type: application/json; charset=utf-8
    Preview: {"hardware":{"shortname":"UDM"},"mac":"74ACB93D0FFB","directConnectDomain":"74acb93d0ffb04aedd4204e4...
  /api/info: Status 401
  /api/auth/login: Status 401

Port 80:
  /api/self: Status 301
  /api/stat/system: Status 301
  /api/system: Status 301
  /api/info: Status 301
  /api/auth/login: Status 301

Port 8443:
  /api/self: Status 404
  /api/stat/system: Status 404
  /api/system: Status 404
  /api/info: Status 404
  /api/auth/login: Status 404


In [6]:
# Determine which IP to use (prefer one that responded)
target_ip = None
for ip in possible_ips:
    conn_test = test_connectivity(ip)
    if any(conn_test.values()):
        target_ip = ip
        break

if not target_ip:
    target_ip = possible_ips[0]  # Default to first

print(f"Using IP: {target_ip}")
print("Enter your UniFi credentials:")

# Get credentials
username = input("Username: ")
password = getpass.getpass("Password: ")

# Common login endpoints and ports
login_configs = [
    {'port': 443, 'endpoint': '/api/login'},
    {'port': 443, 'endpoint': '/api/auth/login'},
    {'port': 443, 'endpoint': '/proxy/network/api/login'},
    {'port': 8443, 'endpoint': '/api/login'},
    {'port': 8443, 'endpoint': '/api/auth/login'},
    {'port': 80, 'endpoint': '/api/login'},
]

session = None
authenticated = False

for config in login_configs:
    url = f"https://{target_ip}:{config['port']}{config['endpoint']}"
    try:
        test_session = requests.Session()
        # Try different login payload formats
        login_payloads = [
            {'username': username, 'password': password},
            {'username': username, 'password': password, 'strict': True},
        ]
        
        for payload in login_payloads:
            response = test_session.post(
                url, 
                json=payload, 
                verify=False, 
                timeout=5,
                headers={'Content-Type': 'application/json'}
            )
            print(f"\nTrying {url}: Status {response.status_code}")
            
            if response.status_code == 200:
                session = test_session
                authenticated = True
                print("✓ Authentication successful!")
                print(f"Response: {response.text[:200]}")
                break
        
        if authenticated:
            break
            
    except Exception as e:
        print(f"  Error: {e}")
        continue

if not authenticated:
    print("\n⚠ Could not authenticate automatically. The device might:")
    print("  - Require different credentials")
    print("  - Use a different API endpoint")
    print("  - Need a different authentication method")
    print("\nLet's continue with unauthenticated discovery...")


Using IP: 192.168.1.1
Enter your UniFi credentials:


Username:  lou@louspringer.com
Password:  ········



Trying https://192.168.1.1:443/api/login: Status 401

Trying https://192.168.1.1:443/api/login: Status 401
  Error: HTTPSConnectionPool(host='192.168.1.1', port=443): Read timed out. (read timeout=5)

Trying https://192.168.1.1:443/proxy/network/api/login: Status 401

Trying https://192.168.1.1:443/proxy/network/api/login: Status 401

Trying https://192.168.1.1:8443/api/login: Status 404

Trying https://192.168.1.1:8443/api/login: Status 404

Trying https://192.168.1.1:8443/api/auth/login: Status 404

Trying https://192.168.1.1:8443/api/auth/login: Status 404
  Error: HTTPSConnectionPool(host='192.168.1.1', port=80): Max retries exceeded with url: /api/login (Caused by SSLError(SSLError(1, '[SSL: RECORD_LAYER_FAILURE] record layer failure (_ssl.c:1077)')))

⚠ Could not authenticate automatically. The device might:
  - Require different credentials
  - Use a different API endpoint
  - Need a different authentication method

Let's continue with unauthenticated discovery...


In [7]:
# Common UniFi API endpoints to explore
api_endpoints = [
    # System info
    "/api/self",
    "/api/stat/system",
    "/api/system",
    "/api/info",
    "/api/s/default/self",
    "/api/s/default/stat/system",
    
    # Devices
    "/api/s/default/stat/device",
    "/api/s/default/list/device",
    
    # Clients
    "/api/s/default/stat/sta",
    "/api/s/default/list/sta",
    
    # Network info
    "/api/s/default/stat/networkconf",
    "/api/s/default/list/networkconf",
    
    # Sites
    "/api/self/sites",
    "/api/sites",
    
    # Events
    "/api/s/default/stat/event",
    
    # Health
    "/api/s/default/stat/health",
    
    # Users
    "/api/s/default/list/user",
    
    # WLAN
    "/api/s/default/list/wlanconf",
    "/api/s/default/rest/wlanconf",
    
    # Port forwarding
    "/api/s/default/rest/portforward",
    
    # Firewall rules
    "/api/s/default/rest/firewallrule",
    
    # Settings
    "/api/s/default/get/setting",
]

def explore_endpoint(base_url, endpoint, session=None):
    """Try to get data from an endpoint"""
    url = f"{base_url}{endpoint}"
    try:
        if session:
            response = session.get(url, verify=False, timeout=5)
        else:
            response = requests.get(url, verify=False, timeout=5)
        
        if response.status_code == 200:
            try:
                data = response.json()
                return {'success': True, 'data': data, 'raw_size': len(response.text)}
            except:
                return {'success': True, 'data': response.text[:500], 'raw_size': len(response.text)}
        elif response.status_code == 401:
            return {'success': False, 'error': 'Unauthorized - needs auth'}
        elif response.status_code == 404:
            return {'success': False, 'error': 'Not found'}
        else:
            return {'success': False, 'error': f'Status {response.status_code}'}
    except Exception as e:
        return {'success': False, 'error': str(e)}

# Determine base URL
if session and authenticated:
    # Try to determine base URL from successful login
    base_urls = [
        f"https://{target_ip}:443",
        f"https://{target_ip}:8443",
        f"https://{target_ip}",
    ]
else:
    base_urls = [f"https://{target_ip}:443", f"https://{target_ip}:8443", f"https://{target_ip}"]

results = {}
for base_url in base_urls:
    print(f"\n\n=== Exploring {base_url} ===\n")
    for endpoint in api_endpoints:
        result = explore_endpoint(base_url, endpoint, session if authenticated else None)
        if result['success']:
            print(f"✓ {endpoint}")
            if isinstance(result['data'], dict):
                print(f"  Keys: {list(result['data'].keys())[:10]}")
                results[endpoint] = result['data']
            else:
                print(f"  Response: {str(result['data'])[:100]}")
        else:
            if 'Unauthorized' not in result['error']:  # Don't spam unauthorized messages
                print(f"✗ {endpoint}: {result['error']}")

print(f"\n\nFound {len(results)} working endpoints!")




=== Exploring https://192.168.1.1:443 ===

✓ /api/system
  Keys: ['hardware', 'mac', 'directConnectDomain', 'deviceState', 'deviceErrorCode', 'uidb', 'debugEnabled', 'cloudConnected', 'isOrHasGatewayInLAN', 'remoteAccessEnabled']


=== Exploring https://192.168.1.1:8443 ===

✗ /api/self: Not found
✗ /api/stat/system: Not found
✗ /api/system: Not found
✗ /api/info: Not found
✗ /api/s/default/self: Not found
✗ /api/s/default/stat/system: Not found
✗ /api/s/default/stat/device: Not found
✗ /api/s/default/list/device: Not found
✗ /api/s/default/stat/sta: Not found
✗ /api/s/default/list/sta: Not found
✗ /api/s/default/stat/networkconf: Not found
✗ /api/s/default/list/networkconf: Not found
✗ /api/self/sites: Not found
✗ /api/sites: Not found
✗ /api/s/default/stat/event: Not found
✗ /api/s/default/stat/health: Not found
✗ /api/s/default/list/user: Not found
✗ /api/s/default/list/wlanconf: Not found
✗ /api/s/default/rest/wlanconf: Not found
✗ /api/s/default/rest/portforward: Not found
✗ /ap

In [8]:
# Display detailed information from each endpoint
for endpoint, data in results.items():
    print(f"\n{'='*60}")
    print(f"ENDPOINT: {endpoint}")
    print(f"{'='*60}")
    
    if isinstance(data, dict):
        # Try to extract meaningful information
        if 'data' in data:
            items = data['data']
            print(f"Found {len(items)} items")
            if len(items) > 0:
                print("\nFirst item structure:")
                pretty_print(items[0] if isinstance(items, list) else items, max_depth=2)
        elif 'meta' in data:
            print("Response structure:")
            pretty_print(data, max_depth=3)
        else:
            print("Data structure:")
            pretty_print(data, max_depth=3)
    else:
        print(f"Response: {str(data)[:500]}")
    
    print()



ENDPOINT: /api/system
Data structure:
hardware: 
  shortname: UDM
mac: 74ACB93D0FFB
directConnectDomain: 74acb93d0ffb04aedd4204e4063405eba61de.id.ui.direct
deviceState: setup
deviceErrorCode: None
uidb: 
  guid: 59a52bba-e33b-4739-a82d-7e875c8ded0e
  id: 142cd194-8164-46a1-ba48-bef078757393
  images: 
    default: 015d558b57f631225c3ba6a73660d4df
    nopadding: 717d100bc54c02f91dfdb70555e14f02
    topology: 74fa25dd073f2f833e72acd2436d2ec0
    mobile-connection: e35a45247f382d2f952ffb1d7f3450f2
    mobile-internet-connected: 8c5965a0983cadd2eec4679def0aa016
    mobile-no-internet: 4f71689cbabedbcc27b49edce2b9b637
debugEnabled: False
cloudConnected: True
isOrHasGatewayInLAN: True
remoteAccessEnabled: True
hasInternet: True
hasInternetLastChanged: 1761258717248
isSingleUser: False
isSsoEnabled: True



## Step 6: Extract specific useful data

Let's try to extract and display useful information like devices, clients, etc.


In [9]:
# Extract devices
if '/api/s/default/stat/device' in results:
    devices_data = results['/api/s/default/stat/device']
    if isinstance(devices_data, dict) and 'data' in devices_data:
        devices = devices_data['data']
        print(f"\n📱 DEVICES ({len(devices)} found):\n")
        for device in devices:
            name = device.get('name', 'Unknown')
            model = device.get('model', 'Unknown')
            ip = device.get('ip', 'N/A')
            mac = device.get('mac', 'N/A')
            version = device.get('version', 'N/A')
            print(f"  {name}")
            print(f"    Model: {model}, IP: {ip}, MAC: {mac}")
            print(f"    Version: {version}")
            print()

# Extract clients/stations
if '/api/s/default/stat/sta' in results:
    clients_data = results['/api/s/default/stat/sta']
    if isinstance(clients_data, dict) and 'data' in clients_data:
        clients = clients_data['data']
        print(f"\n👥 CONNECTED CLIENTS ({len(clients)} found):\n")
        for client in clients[:20]:  # Show first 20
            hostname = client.get('hostname', 'Unknown')
            ip = client.get('ip', 'N/A')
            mac = client.get('mac', 'N/A')
            essid = client.get('essid', 'N/A')
            print(f"  {hostname}")
            print(f"    IP: {ip}, MAC: {mac}, SSID: {essid}")
        if len(clients) > 20:
            print(f"\n  ... and {len(clients) - 20} more clients")

# Extract system info
if '/api/s/default/stat/system' in results:
    sys_data = results['/api/s/default/stat/system']
    print(f"\n🖥️ SYSTEM INFO:\n")
    pretty_print(sys_data, max_depth=2)

# Extract network configurations
if '/api/s/default/stat/networkconf' in results:
    network_data = results['/api/s/default/stat/networkconf']
    if isinstance(network_data, dict) and 'data' in network_data:
        networks = network_data['data']
        print(f"\n🌐 NETWORKS ({len(networks)} found):\n")
        for network in networks:
            name = network.get('name', 'Unknown')
            purpose = network.get('purpose', 'Unknown')
            ip_subnet = network.get('ip_subnet', 'N/A')
            print(f"  {name} ({purpose})")
            print(f"    Subnet: {ip_subnet}")
            print()

# Extract WLAN configurations
if '/api/s/default/list/wlanconf' in results:
    wlan_data = results['/api/s/default/list/wlanconf']
    if isinstance(wlan_data, dict) and 'data' in wlan_data:
        wlans = wlan_data['data']
        print(f"\n📶 WIRELESS NETWORKS ({len(wlans)} found):\n")
        for wlan in wlans:
            name = wlan.get('name', 'Unknown')
            ssid = wlan.get('name', 'Unknown')  # SSID is often same as name
            enabled = wlan.get('enabled', False)
            security = wlan.get('security', 'Unknown')
            print(f"  {name} (SSID: {ssid})")
            print(f"    Enabled: {enabled}, Security: {security}")
            print()


## Step 7: Try additional discovery endpoints

Let's try some more advanced or less common endpoints.


In [10]:
# Additional endpoints to try
advanced_endpoints = [
    # Dashboard
    "/api/s/default/dashboard",
    "/api/s/default/stat/dashboard",
    
    # Alerts
    "/api/s/default/list/alert",
    "/api/s/default/stat/alarm",
    
    # Settings
    "/api/s/default/get/setting",
    "/api/s/default/get/setup",
    
    # Users and admins
    "/api/self/users",
    "/api/users",
    "/api/s/default/list/user",
    
    # Firewall
    "/api/s/default/list/firewallgroup",
    "/api/s/default/rest/firewallrule",
    
    # DPI
    "/api/s/default/stat/dpi",
    "/api/s/default/stat/dpiapp",
    "/api/s/default/stat/dpicategory",
    
    # Routing
    "/api/s/default/rest/routing",
    
    # Dynamic DNS
    "/api/s/default/rest/dynamicdns",
    
    # Port forwarding
    "/api/s/default/list/portforward",
    
    # VPN
    "/api/s/default/rest/vpntunnel",
    
    # Heatmaps
    "/api/s/default/stat/heatmap",
    
    # Speed test
    "/api/s/default/stat/speed-test",
    
    # Insights
    "/api/s/default/stat/insight",
]

if authenticated and session:
    print("Trying advanced endpoints...\n")
    for endpoint in advanced_endpoints:
        for base_url in base_urls:
            result = explore_endpoint(base_url, endpoint, session)
            if result['success']:
                print(f"✓ {base_url}{endpoint}")
                if isinstance(result['data'], dict):
                    if 'data' in result['data']:
                        count = len(result['data']['data']) if isinstance(result['data']['data'], list) else 'N/A'
                        print(f"  Found data items: {count}")
                    else:
                        print(f"  Keys: {list(result['data'].keys())[:5]}")
                results[endpoint] = result['data']
                break


In [11]:
# Cloud Portal API exploration
print("="*60)
print("CLOUD PORTAL API - OFFICIAL API KEYS")
print("="*60)
print("\n🔑 How to get an OFFICIAL API Key from UniFi Site Manager:\n")
print("1. Go to https://unifi.ui.com/ and log in")
print("2. Look for 'API' in the left navigation menu")
print("3. Click on 'API' section")
print("4. Click 'Create API Key'")
print("5. Copy the key immediately - it's only shown once!")
print("\n📖 Official Documentation:")
print("   https://developer.ui.com/site-manager-api/gettingstarted")
print("\n⚠️  Important Notes:")
print("   - MFA is required for UI accounts (since July 2024)")
print("   - Consider creating a local admin account for API access")
print("   - Local accounts don't require MFA for API calls")
print("\n🔧 API Usage:")
print("   - Base URL: https://api.ui.com/v1/")
print("   - Header: X-API-Key: YOUR_API_KEY")
print("   - Format: application/json")

# Try to discover cloud API endpoints if we can infer them
cloud_base_urls = [
    "https://api.ui.com",
    "https://unifi.ui.com/api",
    "https://api.unifi.ui.com",
]

print("\n" + "="*60)
print("\nNote: Cloud Portal API requires OAuth authentication.")
print("You can extract API tokens from browser network requests,")
print("or use the official Site Manager API: https://developer.ui.com/site-manager-api")
print("\n" + "="*60)

# Also try the proxy endpoints that UDM uses for cloud
print("\nTrying cloud-proxy endpoints on local device...")
cloud_proxy_endpoints = [
    "/proxy/network/api/s/default/stat/device",
    "/proxy/network/api/s/default/stat/sta",
    "/proxy/network/api/s/default/stat/system",
    "/proxy/network/api/self",
]

if target_ip:
    for endpoint in cloud_proxy_endpoints:
        result = explore_endpoint(f"https://{target_ip}:443", endpoint, session if authenticated else None)
        if result['success']:
            print(f"✓ {endpoint}")
            results[endpoint] = result['data']
        elif result['error'] not in ['Unauthorized - needs auth', 'Not found']:
            print(f"✗ {endpoint}: {result['error']}")


CLOUD PORTAL API - OFFICIAL API KEYS

🔑 How to get an OFFICIAL API Key from UniFi Site Manager:

1. Go to https://unifi.ui.com/ and log in
2. Look for 'API' in the left navigation menu
3. Click on 'API' section
4. Click 'Create API Key'
5. Copy the key immediately - it's only shown once!

📖 Official Documentation:
   https://developer.ui.com/site-manager-api/gettingstarted

⚠️  Important Notes:
   - MFA is required for UI accounts (since July 2024)
   - Consider creating a local admin account for API access
   - Local accounts don't require MFA for API calls

🔧 API Usage:
   - Base URL: https://api.ui.com/v1/
   - Header: X-API-Key: YOUR_API_KEY
   - Format: application/json


Note: Cloud Portal API requires OAuth authentication.
You can extract API tokens from browser network requests,
or use the official Site Manager API: https://developer.ui.com/site-manager-api


Trying cloud-proxy endpoints on local device...


## Summary

This notebook has discovered and explored various API endpoints on your UniFi Dream Machine. 

### Available Endpoints Found:


In [12]:
# Summary of all discovered endpoints
print("="*60)
print("DISCOVERY SUMMARY")
print("="*60)
print(f"\nTarget Device: {target_ip}")
print(f"Authentication: {'✓ Authenticated' if authenticated else '✗ Not authenticated'}")
print(f"\nWorking Endpoints ({len(results)}):\n")

for endpoint in sorted(results.keys()):
    data = results[endpoint]
    if isinstance(data, dict):
        if 'data' in data and isinstance(data['data'], list):
            print(f"  {endpoint} - {len(data['data'])} items")
        else:
            print(f"  {endpoint} - {list(data.keys())[:3]}")
    else:
        print(f"  {endpoint}")

print("\n" + "="*60)
print("\n💡 TIPS FOR API ACCESS:")
print("  📍 LOCAL ACCESS: Continue with local IP discovery above")
print("  ☁️  CLOUD ACCESS: Get official API key from unifi.ui.com → API")
print("  📖 DOCS: https://developer.ui.com/site-manager-api")
print("\nYou can now access the 'results' dictionary to explore any endpoint data!")
print("Example: results['/api/system'] or results['/api/s/default/stat/device']")


DISCOVERY SUMMARY

Target Device: 192.168.1.1
Authentication: ✗ Not authenticated

Working Endpoints (1):

  /api/system - ['hardware', 'mac', 'directConnectDomain']


💡 TIPS FOR API ACCESS:
  📍 LOCAL ACCESS: Continue with local IP discovery above
  ☁️  CLOUD ACCESS: Get official API key from unifi.ui.com → API
  📖 DOCS: https://developer.ui.com/site-manager-api

You can now access the 'results' dictionary to explore any endpoint data!
Example: results['/api/system'] or results['/api/s/default/stat/device']


## Step 9: Use Official UniFi Site Manager API Key

Get your API key from https://unifi.ui.com/ → API section → Create API Key


In [13]:
# Use Official UniFi Site Manager API Key
# Get your key from: https://unifi.ui.com/ → API → Create API Key

# You can either input it here or use the input() below
api_key = "z4IyPJkX3lP4PlE64ciS64F-1xiv0Va5"  # Your API key
# Or uncomment to input manually: api_key = input("\nPaste your UniFi Site Manager API Key (or press Enter to skip): ").strip()

import requests
if api_key:
    print("\n" + "="*60)
    print("EXPLORING UNIFI SITE MANAGER API")
    print("="*60)
    
    # Create session with API key
    api_session = requests.Session()
    api_session.headers.update({
        'X-API-Key': api_key,
        'Accept': 'application/json',
        'Content-Type': 'application/json'
    })
    
    # Common Site Manager API endpoints
    site_manager_endpoints = [
        # Sites
        "https://api.ui.com/v1/hosts",
        "https://api.ui.com/v1/sites",
        
        # Devices (may require site ID)
        "https://api.ui.com/v1/devices",
        
        # Networks
        "https://api.ui.com/v1/networks",
        
        # System info
        "https://api.ui.com/v1/system",
    ]
    
    print("\nTrying Site Manager API endpoints...\n")
    api_results = {}
    
    for endpoint in site_manager_endpoints:
        try:
            response = api_session.get(endpoint, timeout=10)
            print(f"{endpoint}")
            print(f"  Status: {response.status_code}")
            
            if response.status_code == 200:
                try:
                    data = response.json()
                    if isinstance(data, dict):
                        if 'data' in data:
                            items = data['data']
                            count = len(items) if isinstance(items, list) else 'N/A'
                            print(f"  ✓ Success - Found {count} items")
                            if isinstance(items, list) and len(items) > 0:
                                print(f"  First item keys: {list(items[0].keys())[:5]}")
                        else:
                            print(f"  ✓ Success - Keys: {list(data.keys())[:5]}")
                        api_results[endpoint] = data
                    else:
                        print(f"  ✓ Success - Response: {str(data)[:100]}")
                except:
                    print(f"  ✓ Success - Raw response: {response.text[:100]}")
            elif response.status_code == 401:
                print(f"  ✗ Unauthorized - Check your API key")
            elif response.status_code == 403:
                print(f"  ✗ Forbidden - API key may not have required permissions")
            else:
                print(f"  ✗ Error {response.status_code}: {response.text[:100]}")
        except requests.exceptions.RequestException as e:
            print(f"  ✗ Connection error: {e}")
        
        print()
    
    if api_results:
        print(f"\n✓ Successfully accessed {len(api_results)} API endpoints!")
        print("\nYou can access the data using: api_results['endpoint_url']")
        print("\nExample:")
        if "https://api.ui.com/v1/sites" in api_results:
            sites_data = api_results["https://api.ui.com/v1/sites"]
            if isinstance(sites_data, dict) and 'data' in sites_data:
                sites = sites_data['data']
                if isinstance(sites, list) and len(sites) > 0:
                    print(f"  sites = api_results['https://api.ui.com/v1/sites']['data']")
                    print(f"  Your sites: {[s.get('name', s.get('id', 'Unknown')) for s in sites[:5]]}")
    else:
        print("\n⚠️  No endpoints returned data. Check:")
        print("  1. Your API key is correct")
        print("  2. Your account has proper permissions")
        print("  3. MFA might be blocking access (try local admin account)")
else:
    print("\nSkipping Site Manager API exploration.")
    print("💡 Get your API key from: https://unifi.ui.com/ → API → Create API Key")



EXPLORING UNIFI SITE MANAGER API

Trying Site Manager API endpoints...

https://api.ui.com/v1/hosts
  Status: 200
  ✓ Success - Found 2 items
  First item keys: ['id', 'hardwareId', 'type', 'ipAddress', 'owner']

https://api.ui.com/v1/sites
  Status: 200
  ✓ Success - Found 2 items
  First item keys: ['siteId', 'hostId', 'meta', 'statistics', 'permission']

https://api.ui.com/v1/devices
  Status: 200
  ✓ Success - Found 1 items
  First item keys: ['hostId', 'devices', 'updatedAt']

https://api.ui.com/v1/networks
  Status: 404
  ✗ Error 404: 404 page not found

https://api.ui.com/v1/system
  Status: 404
  ✗ Error 404: 404 page not found


✓ Successfully accessed 3 API endpoints!

You can access the data using: api_results['endpoint_url']

Example:
  sites = api_results['https://api.ui.com/v1/sites']['data']
  Your sites: ['Unknown', 'Unknown']


## Quick API Test with Your Key

Run the cell above to test your API key, or run this quick test below.


In [14]:
# Quick test with your API key
api_key = "z4IyPJkX3lP4PlE64ciS64F-1xiv0Va5"

print("Testing UniFi Site Manager API with your key...\n")

api_session = requests.Session()
api_session.headers.update({
    'X-API-Key': api_key,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
})

# Try the most common endpoints
test_endpoints = [
    "https://api.ui.com/v1/hosts",
    "https://api.ui.com/v1/sites",
]

cloud_results = {}

for endpoint in test_endpoints:
    try:
        response = api_session.get(endpoint, timeout=10)
        print(f"{endpoint}")
        print(f"  Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"  ✓ SUCCESS!")
            
            if isinstance(data, dict):
                if 'data' in data:
                    items = data['data']
                    count = len(items) if isinstance(items, list) else 'N/A'
                    print(f"  Found {count} items")
                    if isinstance(items, list) and len(items) > 0:
                        print(f"  First item preview:")
                        for key, value in list(items[0].items())[:8]:
                            print(f"    {key}: {value}")
                else:
                    print(f"  Response keys: {list(data.keys())[:10]}")
            
            cloud_results[endpoint] = data
        elif response.status_code == 401:
            print(f"  ✗ Unauthorized - API key may be invalid")
        elif response.status_code == 403:
            print(f"  ✗ Forbidden - API key may not have permissions")
        else:
            print(f"  ✗ Error: {response.text[:200]}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    print()

if cloud_results:
    print(f"\n{'='*60}")
    print(f"✓ Successfully accessed {len(cloud_results)} endpoints!")
    print(f"{'='*60}\n")
    
    # Show detailed results
    for endpoint, data in cloud_results.items():
        print(f"\n📊 {endpoint}:")
        if isinstance(data, dict) and 'data' in data:
            items = data['data']
            if isinstance(items, list):
                for i, item in enumerate(items[:5]):  # Show first 5
                    print(f"\n  Item {i+1}:")
                    for key, value in list(item.items())[:10]:
                        print(f"    {key}: {value}")
                if len(items) > 5:
                    print(f"\n  ... and {len(items) - 5} more items")
    
    print(f"\n💾 Data stored in 'cloud_results' dictionary")
    print(f"   Access with: cloud_results['{list(cloud_results.keys())[0]}']")
else:
    print("\n⚠️  Could not access any endpoints. Please check:")
    print("   1. API key is correct")
    print("   2. Your account has proper permissions")
    print("   3. You're connected to the internet")


Testing UniFi Site Manager API with your key...

https://api.ui.com/v1/hosts
  Status: 200
  ✓ SUCCESS!
  Found 2 items
  First item preview:
    id: 67be1762-80ee-46dd-a25f-dd69d18da8c6
    hardwareId: 0ad4313b-ebc9-40e9-b5fd-f020e99c15e9
    type: network-server
    ipAddress: 73.78.113.14
    owner: True
    isBlocked: False
    registrationTime: 2024-09-20T00:26:41Z
    lastConnectionStateChange: 2025-09-08T22:14:16Z

https://api.ui.com/v1/sites
  Status: 200
  ✓ SUCCESS!
  Found 2 items
  First item preview:
    siteId: 66ecbfa4b5fe233342f37b24
    hostId: 67be1762-80ee-46dd-a25f-dd69d18da8c6
    meta: {'desc': 'Default', 'name': 'default', 'timezone': 'America/Denver'}
    statistics: {'counts': {'criticalNotification': 0, 'gatewayDevice': 0, 'guestClient': 0, 'lanConfiguration': 1, 'offlineDevice': 0, 'offlineGatewayDevice': 0, 'offlineWifiDevice': 0, 'offlineWiredDevice': 0, 'pendingUpdateDevice': 0, 'totalDevice': 0, 'wanConfiguration': 0, 'wifiClient': 0, 'wifiConfiguration':

In [15]:
cloud_results

{'https://api.ui.com/v1/hosts': {'data': [{'id': '67be1762-80ee-46dd-a25f-dd69d18da8c6',
    'hardwareId': '0ad4313b-ebc9-40e9-b5fd-f020e99c15e9',
    'type': 'network-server',
    'ipAddress': '73.78.113.14',
    'owner': True,
    'isBlocked': False,
    'registrationTime': '2024-09-20T00:26:41Z',
    'lastConnectionStateChange': '2025-09-08T22:14:16Z',
    'latestBackupTime': '',
    'userData': {'permissions': {'network.management': ['admin']},
     'status': 'ACTIVE'},
    'reportedState': {'controller_uuid': '0ad4313b-ebc9-40e9-b5fd-f020e99c15e9',
     'firmware_version': None,
     'hardware_id': '0ad4313b-ebc9-40e9-b5fd-f020e99c15e9',
     'host_type': 0,
     'hostname': 'example-domain.ui.com',
     'inform_port': 8080,
     'ipAddrs': ['192.168.12.156'],
     'mgmt_port': 8443,
     'name': 'UniFi Network',
     'override_inform_host': False,
     'release_channel': 'release',
     'state': 'disconnected',
     'version': '8.4.62'}},
   {'id': '74ACB93D0FFB0000000004AEDD4200

## Create DataFrames from API Data

Convert the API responses into pandas DataFrames for easier analysis and visualization.


In [16]:
from pandas import DataFrame as pd
# Helper function to convert API data to DataFrame
def create_dataframe(data, name="data"):
    """
    Convert API response data into a pandas DataFrame
    
    Args:
        data: API response data (dict with 'data' key, or list, or dict)
        name: Name for the DataFrame (for display purposes)
    """
    if isinstance(data, dict):
        if 'data' in data:
            items = data['data']
        else:
            # Try to find a list in the dict values
            items = [data]
    elif isinstance(data, list):
        items = data
    else:
        return None
    
    if not items or len(items) == 0:
        print(f"No data to convert for {name}")
        return None
    
    if isinstance(items, list) and len(items) > 0:
        try:
            df = pd.DataFrame(items)
            print(f"✓ Created DataFrame '{name}' with {len(df)} rows and {len(df.columns)} columns")
            return df
        except Exception as e:
            print(f"✗ Error creating DataFrame for {name}: {e}")
            return None
    else:
        return None

# Create DataFrames from cloud_results if available
if 'cloud_results' in globals() and cloud_results:
    print("="*60)
    print("CREATING DATAFRAMES FROM API DATA")
    print("="*60)
    print()
    
    # Dictionary to store all DataFrames
    dfs = {}
    
    # Sites/Hosts DataFrame
    if "https://api.ui.com/v1/hosts" in cloud_results:
        df_hosts = create_dataframe(cloud_results["https://api.ui.com/v1/hosts"], "hosts")
        if df_hosts is not None:
            dfs['hosts'] = df_hosts
            print(f"\nHosts DataFrame preview:")
            print(df_hosts.head())
            print(f"\nHosts DataFrame info:")
            print(df_hosts.info())
    
    if "https://api.ui.com/v1/sites" in cloud_results:
        df_sites = create_dataframe(cloud_results["https://api.ui.com/v1/sites"], "sites")
        if df_sites is not None:
            dfs['sites'] = df_sites
            print(f"\nSites DataFrame preview:")
            print(df_sites.head())
            print(f"\nSites DataFrame info:")
            print(df_sites.info())
    
    print(f"\n✓ Created {len(dfs)} DataFrame(s)")
    print(f"Available DataFrames: {list(dfs.keys())}")
else:
    print("⚠️  No cloud_results found. Run the API cells above first.")
    dfs = {}


CREATING DATAFRAMES FROM API DATA

✗ Error creating DataFrame for hosts: type object 'DataFrame' has no attribute 'DataFrame'
✗ Error creating DataFrame for sites: type object 'DataFrame' has no attribute 'DataFrame'

✓ Created 0 DataFrame(s)
Available DataFrames: []


## Fetch More Data and Create Additional DataFrames

Let's fetch more detailed data from the API and create DataFrames for devices, clients, networks, etc.


In [17]:
# Fetch more detailed data from API
api_key = "z4IyPJkX3lP4PlE64ciS64F-1xiv0Va5"

api_session = requests.Session()
api_session.headers.update({
    'X-API-Key': api_key,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
})

# Extended list of endpoints to explore
extended_endpoints = [
    "https://api.ui.com/v1/hosts",
    "https://api.ui.com/v1/sites",
    "https://api.ui.com/v1/devices",
    "https://api.ui.com/v1/networks",
    "https://api.ui.com/v1/clients",
    "https://api.ui.com/v1/events",
    "https://api.ui.com/v1/system",
]

print("Fetching extended data from Site Manager API...\n")
extended_data = {}

for endpoint in extended_endpoints:
    try:
        response = api_session.get(endpoint, timeout=10)
        if response.status_code == 200:
            data = response.json()
            extended_data[endpoint] = data
            print(f"✓ {endpoint.split('/')[-1]}")
        else:
            print(f"✗ {endpoint.split('/')[-1]}: {response.status_code}")
    except Exception as e:
        print(f"✗ {endpoint.split('/')[-1]}: {e}")

print(f"\n✓ Fetched data from {len(extended_data)} endpoints")


Fetching extended data from Site Manager API...

✓ hosts
✓ sites
✓ devices
✗ networks: 404
✗ clients: 404
✗ events: 404
✗ system: 404

✓ Fetched data from 3 endpoints


In [18]:
# Create DataFrames for all available data types
print("="*60)
print("CREATING DATAFRAMES FROM ALL DATA")
print("="*60)
print()

all_dfs = {}

# Map endpoint to DataFrame name
endpoint_map = {
    "https://api.ui.com/v1/hosts": "hosts",
    "https://api.ui.com/v1/sites": "sites",
    "https://api.ui.com/v1/devices": "devices",
    "https://api.ui.com/v1/networks": "networks",
    "https://api.ui.com/v1/clients": "clients",
    "https://api.ui.com/v1/events": "events",
    "https://api.ui.com/v1/system": "system",
}

for endpoint, data in extended_data.items():
    df_name = endpoint_map.get(endpoint, endpoint.split('/')[-1])
    df = create_dataframe(data, df_name)
    if df is not None:
        all_dfs[df_name] = df

print(f"\n{'='*60}")
print(f"SUMMARY: Created {len(all_dfs)} DataFrames")
print(f"{'='*60}\n")

for df_name, df in all_dfs.items():
    print(f"📊 {df_name.upper()}")
    print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
    print(f"   Column names: {', '.join(list(df.columns)[:10])}")
    if len(df.columns) > 10:
        print(f"   ... and {len(df.columns) - 10} more columns")
    print()

print("\n💾 All DataFrames stored in 'all_dfs' dictionary")
print("   Access with: all_dfs['hosts'], all_dfs['devices'], etc.")


CREATING DATAFRAMES FROM ALL DATA

✗ Error creating DataFrame for hosts: type object 'DataFrame' has no attribute 'DataFrame'
✗ Error creating DataFrame for sites: type object 'DataFrame' has no attribute 'DataFrame'
✗ Error creating DataFrame for devices: type object 'DataFrame' has no attribute 'DataFrame'

SUMMARY: Created 0 DataFrames


💾 All DataFrames stored in 'all_dfs' dictionary
   Access with: all_dfs['hosts'], all_dfs['devices'], etc.


In [22]:
all_dfs

{}

## Display DataFrames

Let's display and explore each DataFrame.


In [19]:
# Display each DataFrame with detailed information
for df_name, df in all_dfs.items():
    print("="*60)
    print(f"DATAFRAME: {df_name.upper()}")
    print("="*60)
    print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns\n")
    
    # Display first few rows
    print("First 5 rows:")
    print(df.head())
    
    # Show column data types
    print(f"\n\nColumn Data Types:")
    print(df.dtypes)
    
    # Show summary statistics for numeric columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_cols) > 0:
        print(f"\n\nSummary Statistics (numeric columns):")
        print(df[numeric_cols].describe())
    
    # Show unique counts for categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'bool']).columns[:10]  # First 10
    if len(categorical_cols) > 0:
        print(f"\n\nUnique Values (first 10 categorical columns):")
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"  {col}: {unique_count} unique values")
            if unique_count <= 10:
                print(f"    Values: {list(df[col].unique())}")
    
    print("\n" + "="*60 + "\n")


## Useful DataFrame Operations

Some common operations you can perform on the DataFrames.


In [20]:
# Example DataFrame operations

print("📊 Available DataFrames:", list(all_dfs.keys()))
print("\nExample operations:\n")

# Show how to access DataFrames
if 'hosts' in all_dfs:
    print("1. Access hosts DataFrame:")
    print("   all_dfs['hosts']")
    print("   or: df_hosts = all_dfs['hosts']")
    print()

# Example: Filter and search
if 'devices' in all_dfs:
    df_devices = all_dfs['devices']
    print("2. Filter devices:")
    print("   # Find devices by name")
    print("   df_devices[df_devices['name'].str.contains('Dream', na=False)]")
    print()

# Example: Group by operations
if 'clients' in all_dfs:
    print("3. Group clients:")
    print("   # Count clients by SSID")
    print("   all_dfs['clients'].groupby('essid').size()")
    print()

# Example: Select specific columns
print("4. Select columns:")
print("   df[['name', 'ip', 'mac']]  # Select specific columns")
print()

# Example: Export to CSV
print("5. Export to CSV:")
print("   all_dfs['hosts'].to_csv('unifi_hosts.csv', index=False)")
print("   all_dfs['devices'].to_csv('unifi_devices.csv', index=False)")
print()

# Quick summary of what's in each DataFrame
print("\n📋 QUICK SUMMARY:\n")
for df_name, df in all_dfs.items():
    print(f"{df_name}:")
    # Show a few interesting columns if they exist
    interesting_cols = ['name', 'ip', 'mac', 'model', 'version', 'status', 'id']
    found_cols = [col for col in interesting_cols if col in df.columns]
    if found_cols:
        print(f"  Key columns: {', '.join(found_cols)}")
    print(f"  Total items: {len(df)}")
    print()


📊 Available DataFrames: []

Example operations:

4. Select columns:
   df[['name', 'ip', 'mac']]  # Select specific columns

5. Export to CSV:
   all_dfs['hosts'].to_csv('unifi_hosts.csv', index=False)
   all_dfs['devices'].to_csv('unifi_devices.csv', index=False)


📋 QUICK SUMMARY:



In [21]:
results

{'/api/system': {'hardware': {'shortname': 'UDM'},
  'mac': '74ACB93D0FFB',
  'directConnectDomain': '74acb93d0ffb04aedd4204e4063405eba61de.id.ui.direct',
  'deviceState': 'setup',
  'deviceErrorCode': None,
  'uidb': {'guid': '59a52bba-e33b-4739-a82d-7e875c8ded0e',
   'id': '142cd194-8164-46a1-ba48-bef078757393',
   'images': {'default': '015d558b57f631225c3ba6a73660d4df',
    'nopadding': '717d100bc54c02f91dfdb70555e14f02',
    'topology': '74fa25dd073f2f833e72acd2436d2ec0',
    'mobile-connection': 'e35a45247f382d2f952ffb1d7f3450f2',
    'mobile-internet-connected': '8c5965a0983cadd2eec4679def0aa016',
    'mobile-no-internet': '4f71689cbabedbcc27b49edce2b9b637'}},
  'debugEnabled': False,
  'cloudConnected': True,
  'isOrHasGatewayInLAN': True,
  'remoteAccessEnabled': True,
  'hasInternet': True,
  'hasInternetLastChanged': 1761258717248,
  'isSingleUser': False,
  'isSsoEnabled': True}}